# Clean Markowitz / Risk-Return Notebook

This notebook builds:
- `returns_wide`: cumulative returns per ticker per horizon
- `stddev_wide`: std dev of daily returns per ticker per horizon
- `sharpe_wide`: annualized Sharpe ratio per ticker per horizon

Then it plots:
- Risk/return scatter (colored by horizon)
- Sharpe ratio grouped bar chart (colored by horizon)

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from returns import (
    get_returns_table,
    get_stddev_table,
    get_sharpe_table,
)

In [10]:
# Load the price panel (generated by a) data_pull.ipynb)

from datetime import datetime, timedelta

yesterday = (datetime.today() - timedelta(days=2)).strftime('%Y-%m-%d')
yesterday = yesterday.replace("-", "_")

df = pd.read_parquet(f"ETFs_data_{yesterday}.parquet.gzip")
df.rename_axis('date', inplace=True)
df.head()

,Price,High,Low,Open,Volume,Stock_name,Close
date,,,,,,,
2022-07-26,22.240000,22.490000,22.190001,22.330000,16800.0,BRK,NaN
2022-07-27,22.580000,22.670000,22.260000,22.360001,110700.0,BRK,NaN
2022-07-28,22.900000,22.910000,22.309999,22.719999,38500.0,BRK,NaN
2022-07-29,23.450001,23.559999,23.070000,23.240000,53400.0,BRK,NaN
2022-08-02,22.730000,23.129999,22.730000,23.129999,34200.0,BRK,NaN


In [11]:
all_tickers = sorted(df['Stock_name'].unique())
all_tickers[:10]

['BRK',
 'BXF.TO',
 'CBD.TO',
 'CBH.TO',
 'CBO.TO',
 'CDZ.TO',
 'CEW.TO',
 'CGL.TO',
 'CGR.TO',
 'CHB.TO']

In [12]:
[x for x in list(set(df['Stock_name'])) if 'XEQ' in x]

['XEQT.TO']

In [13]:
# Pick a small subset to start (edit freely)
patterns = ['VFV', 'XUU', 'VOO', 'BRK', 'QQ', 'XEF', 'XEQ']
subset_tickers = sorted({t for t in all_tickers if any(p in t for p in patterns)})
subset_tickers

['BRK',
 'QQC.TO',
 'QQQ',
 'VFV',
 'VFV.TO',
 'XEF.TO',
 'XEQT.TO',
 'XQQ.TO',
 'XUU.TO',
 'ZQQ.TO']

In [14]:
subset_df = df[df['Stock_name'].isin(subset_tickers)]
subset_df.head()

,Price,High,Low,Open,Volume,Stock_name,Close
date,,,,,,,
2022-07-26,22.240000,22.490000,22.190001,22.330000,16800.0,BRK,NaN
2022-07-27,22.580000,22.670000,22.260000,22.360001,110700.0,BRK,NaN
2022-07-28,22.900000,22.910000,22.309999,22.719999,38500.0,BRK,NaN
2022-07-29,23.450001,23.559999,23.070000,23.240000,53400.0,BRK,NaN
2022-08-02,22.730000,23.129999,22.730000,23.129999,34200.0,BRK,NaN


In [15]:
horizons = ['6M', '1Y', '3Y', '5Y']

returns_wide = get_returns_table(subset_df, subset_tickers, horizons)
stddev_wide  = get_stddev_table(subset_df, subset_tickers, horizons)

# Set risk_free_rate_annual to what you want (e.g. 0.03 for 3%)
sharpe_wide  = get_sharpe_table(subset_df, subset_tickers, horizons, risk_free_rate_annual=0.0)

returns_wide

,6M,1Y,3Y,5Y
Ticker,,,,
BRK,0.035133,0.040781,0.476171,0.629496
QQC.TO,0.133814,0.174831,1.314112,1.188984
QQQ,-0.035070,0.061096,1.070722,1.164640
VFV,0.132405,0.141470,0.865549,1.131977
VFV.TO,0.132405,0.141470,0.865549,1.131977
XEF.TO,0.154719,0.296568,0.590307,0.659930
XEQT.TO,0.159110,0.232042,0.720163,0.884397
XQQ.TO,0.082333,0.165917,1.095743,0.803238
XUU.TO,0.130867,0.131731,0.822243,1.036895


In [16]:
stddev_wide

,6M,1Y,3Y,5Y
Ticker,,,,
BRK,0.007982,0.011563,0.009919,0.010720
QQC.TO,0.009284,0.014396,0.012118,0.013274
QQQ,0.013311,0.012526,0.015929,0.015571
VFV,0.006696,0.011596,0.008864,0.009377
VFV.TO,0.006696,0.011596,0.008864,0.009377
XEF.TO,0.006647,0.009442,0.007651,0.008191
XEQT.TO,0.006229,0.009698,0.007551,0.008114
XQQ.TO,0.009540,0.014531,0.012491,0.014373
XUU.TO,0.007029,0.011904,0.009120,0.009677


In [17]:
sharpe_wide

,6M,1Y,3Y,5Y
Ticker,,,,
BRK,0.564278,0.222170,0.880343,0.602803
QQC.TO,1.937379,0.765009,1.677543,0.805028
QQQ,-0.326105,0.307256,1.085931,0.675705
VFV,2.656360,0.768544,1.641819,1.098172
VFV.TO,2.656360,0.768544,1.641819,1.098172
XEF.TO,3.159624,1.978647,1.376882,0.820402
XEQT.TO,3.474048,1.507312,1.653394,1.048820
XQQ.TO,1.132030,0.719259,1.410655,0.548508
XUU.TO,2.499197,0.697122,1.529446,0.995398


## Plot: Risk/Return scatter (Plotly)
Each horizon is a different color.

In [18]:
def plot_risk_return_scatter_by_horizon(returns_wide, stddev_wide, horizons, title=None, width=900, height=600):
    horizons = list(horizons)

    ret_long = returns_wide[horizons].reset_index()
    ret_long = ret_long.rename(columns={ret_long.columns[0]: 'ticker'}).melt(
        id_vars='ticker', var_name='horizon', value_name='return'
    )

    vol_long = stddev_wide[horizons].reset_index()
    vol_long = vol_long.rename(columns={vol_long.columns[0]: 'ticker'}).melt(
        id_vars='ticker', var_name='horizon', value_name='stddev'
    )

    plot_df = ret_long.merge(vol_long, on=['ticker', 'horizon']).dropna()

    fig = px.scatter(
        plot_df,
        x='stddev',
        y='return',
        color='horizon',
        text='ticker',
        title=title or 'Risk/Return Scatter by Horizon',
    )
    fig.update_traces(textposition='top center', marker=dict(size=10, opacity=0.85))
    fig.update_layout(
        width=width,
        height=height,
        xaxis_title='Std Dev of Daily Returns',
        yaxis_title='Return',
        legend_title='Horizon',
    )
    return fig

fig = plot_risk_return_scatter_by_horizon(returns_wide, stddev_wide, horizons, width=1000, height=650)
fig.show()

## Plot: Sharpe ratio grouped bar chart (Plotly)

In [19]:
plot_df = (
    sharpe_wide
    .reset_index()
    .rename(columns={sharpe_wide.reset_index().columns[0]: 'ticker'})
    .melt(id_vars='ticker', var_name='horizon', value_name='sharpe')
    .dropna()
)

fig = px.bar(
    plot_df,
    x='ticker',
    y='sharpe',
    color='horizon',
    barmode='group',
    title='Sharpe Ratio by Ticker and Horizon',
)
fig.update_layout(
    width=1100,
    height=550,
    xaxis_title='Ticker',
    yaxis_title='Sharpe Ratio',
    legend_title='Horizon',
)
fig.show()